<a href="https://colab.research.google.com/github/krishna1072/JanataHack-Recommendation-Systems/blob/master/RNN_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Libraries
import pandas as pd
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,Input,BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


In [ ]:
# Load train data
zf = zipfile.ZipFile('/train_mddNHeX.zip') # having First.csv zipped file.
train = pd.read_csv(zf.open('train.csv'))
display(train.head())
train.info()

# Load all the challenges
challenges = pd.read_csv(zf.open('challenge_data.csv'))
challenges['publish_date'] = pd.to_datetime(challenges['publish_date'])
display(challenges.head())
challenges.info()

# Load test data
zf = zipfile.ZipFile('/test_HLxMpl7.zip') # having First.csv zipped file.
test = pd.read_csv(zf.open('test.csv'))
display(test.head())
test.info()

,user_sequence,user_id,challenge_sequence,challenge
0,4576_1,4576,1,CI23714
1,4576_2,4576,2,CI23855
2,4576_3,4576,3,CI24917
3,4576_4,4576,4,CI23663
4,4576_5,4576,5,CI23933


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903916 entries, 0 to 903915
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   user_sequence       903916 non-null  object
 1   user_id             903916 non-null  int64 
 2   challenge_sequence  903916 non-null  int64 
 3   challenge           903916 non-null  object
dtypes: int64(2), object(2)
memory usage: 27.6+ MB


,challenge_ID,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,CI23478,2,SI2445,37.0,2006-06-05,AI563576,M,AOI100001,NaN
1,CI23479,2,SI2435,48.0,2002-10-17,AI563577,M,AOI100002,32.0
2,CI23480,1,SI2435,15.0,2002-10-16,AI563578,M,AOI100003,NaN
3,CI23481,1,SI2710,236.0,2003-09-19,AI563579,M,AOI100004,70.0
4,CI23482,2,SI2440,137.0,2002-03-21,AI563580,M,AOI100005,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5606 entries, 0 to 5605
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   challenge_ID          5606 non-null   object        
 1   programming_language  5606 non-null   int64         
 2   challenge_series_ID   5594 non-null   object        
 3   total_submissions     5254 non-null   float64       
 4   publish_date          5606 non-null   datetime64[ns]
 5   author_ID             5567 non-null   object        
 6   author_gender         5509 non-null   object        
 7   author_org_ID         5358 non-null   object        
 8   category_id           3765 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 394.3+ KB


,user_sequence,user_id,challenge_sequence,challenge
0,4577_1,4577,1,CI23855
1,4577_2,4577,2,CI23933
2,4577_3,4577,3,CI24917
3,4577_4,4577,4,CI24915
4,4577_5,4577,5,CI23714


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397320 entries, 0 to 397319
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   user_sequence       397320 non-null  object
 1   user_id             397320 non-null  int64 
 2   challenge_sequence  397320 non-null  int64 
 3   challenge           397320 non-null  object
dtypes: int64(2), object(2)
memory usage: 12.1+ MB


In [ ]:
# Create labels
label = train[train.challenge_sequence > 10][['user_id','challenge']]
label.rename(columns={'challenge':'label'},inplace=True)

In [ ]:
# Treat the sequence of challenges as text
df = train[train.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()

In [ ]:
# Merge Labels
df = df.merge(label)

In [ ]:
# Validation split for early stopping
df_train, df_validation = train_test_split(df.sample(frac=1,random_state=123), test_size=0.05, random_state=123)

In [ ]:
# Encode challenges
encoder = LabelEncoder()
encoder.fit(challenges['challenge_ID'])
df_train['brand_id_encoded'] = encoder.transform(df_train.label)
df_validation['brand_id_encoded'] = encoder.transform(df_validation.label)

In [ ]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['challenge'])

In [ ]:
# Constants
NB_WORDS = len(tokenizer.word_index)
MAX_SEQUENCE_LENGTH = 10
N_CATEGORIES = challenges.shape[0]

In [ ]:
# Create sequences
sequences_train = tokenizer.texts_to_sequences(df_train['challenge'])
sequences_validation = tokenizer.texts_to_sequences(df_validation['challenge'])

In [ ]:
# Pad sequences
x_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
x_validation = pad_sequences(sequences_validation, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

In [ ]:
# Set Labels
y_train = df_train['brand_id_encoded'].values
y_validation= df_validation['brand_id_encoded'].values

In [ ]:
# NN architecture
def get_model(path='',lr=0.001):
    adam = Adam(lr=lr)
    inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    x = Embedding(NB_WORDS,256)(inp)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1))(x)
    x = Dropout(0.4)(x)
    x = Dense(N_CATEGORIES, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    if path != '':
        model.load_weights(path)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
# Initialize the model
model = get_model()

In [ ]:
# Model callbacks
path = 'best_model_weights'
es_callback = EarlyStopping(monitor="val_loss", patience=5)
mc_callback = ModelCheckpoint('{}.hdf5'.format(path), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks = [es_callback,mc_callback]

In [ ]:
# Fit the model
model.fit(x_train,
          y_train,
          epochs=100,
          batch_size=1024,
          validation_data=(x_validation, y_validation),
          callbacks = callbacks
)

Train on 198166 samples, validate on 10430 samples
Epoch 1/100
198166/198166 [==============================] - 11s 57us/step - loss: 3.4948 - accuracy: 0.1646 - val_loss: 4.3845 - val_accuracy: 0.0758
Epoch 2/100
198166/198166 [==============================] - 11s 55us/step - loss: 3.4630 - accuracy: 0.1668 - val_loss: 4.3956 - val_accuracy: 0.0766
Epoch 3/100
198166/198166 [==============================] - 11s 56us/step - loss: 3.4342 - accuracy: 0.1699 - val_loss: 4.4080 - val_accuracy: 0.0748
Epoch 4/100
198166/198166 [==============================] - 11s 55us/step - loss: 3.4096 - accuracy: 0.1706 - val_loss: 4.4158 - val_accuracy: 0.0739
Epoch 5/100
198166/198166 [==============================] - 11s 55us/step - loss: 3.3864 - accuracy: 0.1733 - val_loss: 4.4321 - val_accuracy: 0.0719
Epoch 6/100
198166/198166 [==============================] - 11s 55us/step - loss: 3.3654 - accuracy: 0.1752 - val_loss: 4.4436 - val_accuracy: 0.0739


In [ ]:
# Load best weights
model = get_model('{}.hdf5'.format(path))

In [ ]:
# Test preprocessing
def padding(text):
	return pad_sequences(tokenizer.texts_to_sequences(text), maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
test_text = test[test.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()
x_test = padding(test_text.challenge)

In [ ]:
# Get top 3 predictions for each user
pred = model.predict(x_test,batch_size=2048)
pred = pred.argsort(axis=1)[:,-3:][:,::-1]

In [ ]:
# Write Predictions
df_list = []
for i in range(3):
	test_11 = test_text[['user_id']]
	test_11['user_sequence'] = test_11.user_id.astype(str) + '_'+str(i+11)
	test_11['challenge'] = encoder.inverse_transform(pred[:,i])
	df_list.append(test_11[['user_sequence','challenge']])
pd.concat(df_list).to_csv('/RNN_submission.csv',index=False)